In [2]:
import pandas as pd 
import numpy as np

In [15]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return (x * (1-x))

def relu(x):
    for i in range(x.shape[1]):
        x[0][i] = max(0, x[0][i])
    return x

def derivative_relu(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x



In [4]:
input_layer_size = 2
hidden_layer_size = 10
output_layer_size = 1 

alpha = 0.01 
batch_size = 10

In [32]:
data_test = pd.read_excel("HW3Avalidate.xlsx") 
data_train = pd.read_excel("HW3Atrain.xlsx")

X_train = data_train.copy()
X_train = X_train.drop('y', axis=1)
Y_train = data_train.copy()
Y_train = Y_train.drop('X_0', axis=1)
Y_train = Y_train.drop('X_1', axis=1)

X_test = data_test.copy()
X_test = X_test.drop('y', axis=1)
Y_test = data_test.copy()
Y_test = Y_test.drop('X_0', axis=1)
Y_test = Y_test.drop('X_1', axis=1)


x_0_min = min(X_train['X_0'])
x_0_max = max(X_train['X_0'])

x_1_min = min(X_train['X_1'])
x_1_max = max(X_train['X_1'])

test_x_0_min = min(X_test['X_0'])
test_x_0_max = max(X_test['X_0'])

test_x_1_min = min(X_test['X_1'])
test_x_1_max = max(X_test['X_1'])

for i in range(len(X_train['X_0'])):
    X_train['X_0'][i] = (X_train['X_0'][i]-x_0_min)/(x_0_max-x_0_min) #formula used for Min-Max
    X_train['X_1'][i] = (X_train['X_1'][i]-x_1_min)/(x_1_max-x_1_min) #formula used for Min-Max
    
for i in range(len(X_test['X_0'])):
    X_test['X_0'][i] = (X_test['X_0'][i]-x_0_min)/(x_0_max-x_0_min) #formula used for Min-Max
    X_test['X_1'][i] = (X_test['X_1'][i]-x_1_min)/(x_1_max-x_1_min) #formula used for Min-Max

training_data = []
for i in range(len(X_train['X_0'])):
    training_data.append(np.array([X_train['X_0'][i],X_train['X_1'][i]]))
    
testing_data = []
for i in range(len(X_test['X_0'])):
    testing_data.append(np.array([X_test['X_0'][i],X_test['X_1'][i]]))
#print(testing_data)
print(Y_train)


     y
0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
8    0
9    1
10   1
11   0
12   0
13   1
14   0
15   0
16   1
17   1
18   1
19   0
20   0
21   1
22   1
23   0
24   0
25   0
26   0
27   0
28   1
29   1
..  ..
380  0
381  0
382  0
383  0
384  0
385  0
386  0
387  1
388  0
389  0
390  0
391  1
392  0
393  1
394  1
395  0
396  1
397  1
398  0
399  1
400  1
401  1
402  1
403  0
404  1
405  1
406  1
407  0
408  0
409  1

[410 rows x 1 columns]


In [81]:
class Network_layer:
    weights = []
    bias = []
    def __init__(self, dim1, dim2):
        #self.size = size
        self.weights = np.random.rand(dim1,dim2) * 0.1
        self.bias = np.zeros((1,dim2))
    

class Neaural_net:
    hidden_layers = []
    def __init__(self, shape):
        input_layer = np.zeros((1, shape[0]))
        for i in range(1, len(shape)):
            self.hidden_layers.append(Network_layer(shape[i-1], shape[i]))
        output_layer = np.zeros((1, shape[len(shape)-1]))
    
    def forward_pass(self, X):
        curr_input = X
        phi_arr = []
        for i in range(len(self.hidden_layers)):
            #print(curr_input.shape, self.hidden_layers[i].weights.shape)
            h = np.dot(curr_input, self.hidden_layers[i].weights) + self.hidden_layers[i].bias
            if i != len(self.hidden_layers) - 1:
                phi = relu(h)[0]
                #print(h)
                curr_input = phi
            else:
                phi = sigmoid(h)[0]
                curr_input = phi
                
            phi_arr.append(phi)
            #print(phi_arr)
        
        output = phi
        
        return phi_arr

        
    def back_propigation():
        print("temp")
    
    #Skoda seinna
    def get_value_from_pred(self, pred):
        #diff = pred[1] - pred[0]
        #return diff
        
        if pred[0] > pred[1]:
            return 0
        else:
            return 1
        
    def compare_errors(self, pred, y):
        if pred == y:
            return 0
        else:
            return 1
        
    def testing(self):
        error_count = 0
        for i in range(len(X_test)):
            pred = self.forward_pass(X_test)
            error_count = error_count + self.compare_errors(self.get_value_from_pred(pred[len(pred)-1]), Y_test['y'][i])
            error_percent = (len(X_test) - error_count)/len(X_test)
        print(error_percent)
        print()
    
    def train(self, X_train):
        for j in range(1000):
            error_count = 0
            for i in range(len(X_train)):
                pred = self.forward_pass(X_train[i])
                #print(pred)
                #print(Y_train['y'][i])

                #error_count = error_count + self.compare_errors(self.get_value_from_pred(pred[len(pred)-1]), Y_train['y'][i])
                error_3 = Y_train['y'][i] - self.get_value_from_pred(pred[len(pred)-1])
                delta_3 = error_3 * derivative_sigmoid(pred[len(pred)-1])

                error_2 = delta_3.dot(self.hidden_layers[2].weights.T)
                delta_2 = error_2 * derivative_relu(pred[len(pred)-2])

                error_1 = delta_2.dot(self.hidden_layers[1].weights.T)
                delta_1 = error_1 * derivative_relu(pred[len(pred)-3])
                #print(X_train[i][:,None].shape, delta_1[:,None].T.shape)
                layer1_adjustment = X_train[i][:,None].dot(delta_1[:,None].T)
                layer2_adjustment = pred[len(pred)-3].T.dot(delta_2)
                #print(pred[len(pred)-2].T.shape, delta_3.shape)
                layer3_adjustment = pred[len(pred)-2][:,None].dot(delta_3[:,None].T)

                # Adjust the weights.
                self.hidden_layers[0].weights += layer1_adjustment
                self.hidden_layers[1].weights += layer2_adjustment
                self.hidden_layers[2].weights += layer3_adjustment
                #print(self.hidden_layers[2].weights)
                #print()
                #error_percent = (len(X_train) - error_count)/len(X_train)
            if(j > 1):
                    print(j)
                    #print(self.get_value_from_pred(pred[len(pred)-1]), Y_train['y'][i])
                    #print()

a = np.random.randn(2,10)
b = np.random.randn(10,10)
d = np.dot(a,b)
#print(d)
        
shape = [2,10,10,2]
NN = Neaural_net(shape)
#print([X_train['X_0'][0],X_train['X_1'][0]])
#x = [X_train['X_0'][0],X_train['X_1'][0]]
#test = np.asarray(x)
NN.train(training_data)
NN.testing()

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [28]:
a = np.array([1,2])
b = np.array([1,2,3,4,5,6,7,8,9,10])
b = np.random.rand(a.shape[0],10)
print(b.shape)
print(np.dot(a,b))


(2, 10)
[0.92255083 2.05405561 1.53219312 1.75838565 1.26657119 2.05530305
 1.00730676 1.33028476 1.79629828 1.34743652]


In [36]:
X_test = pd.read_excel("HW3Avalidate.xlsx") 
X_train = pd.read_excel("HW3Atrain.xlsx") 

'''
h0 = np.dot(X, weights1) + bias1
phi1 = relu(h0)

h1 = np.dot(phi1, weights2) + bias2
phi2 = relu(h1)

h2 = np.dot(phi2, weights3) + bias3
phi3 = relu(h2)

h3 = np.dot(phi3, weights4) + bias4
phi4 = sigmoid(h3)
'''

print(X_train)

             X_0          X_1  y
0    2917.211242  3289.522533  0
1    1888.937716   781.528356  0
2    4188.521414  1554.476261  0
3    8145.555339  9804.066728  0
4    9584.488981  6176.337189  0
5    4039.633757   167.607143  0
6    2399.020781  6251.891672  1
7    5899.876835  7732.514778  0
8    1832.517788  1300.730270  0
9    1612.288975  6426.781763  1
10   9286.626998  1562.956170  1
11   8497.141947  8806.179833  0
12   9355.215737  9845.547425  0
13   6959.671139  7096.002717  1
14   8318.774203  6561.487762  0
15   2956.353881  1744.414739  0
16   3494.320864  7175.690252  1
17   9733.605268   222.444205  1
18   3357.382654  9136.605932  1
19   1466.794269  7637.730334  0
20   2309.226297  2891.563126  0
21   1640.887579  9801.869658  1
22   5781.556560   248.504846  1
23   3828.052410  2037.851361  0
24    951.195475  2630.862744  0
25   6948.834627  7701.470309  0
26   7196.243632  9244.761921  0
27    848.571206  1321.366007  0
28    513.550491  9111.127856  1
29   3146.

## 1. (10 points) Activation and Loss functions. 
Please choose suitable activation functions φ(0), φ(1), φ(2) and a suitable Loss function to perform the task. Report and justify your choices in the report.

## 2. (10 points) Learning rate, batch size, initialization. 
Please choose a suitable learning rate, batch size, initialization of the parameter values, and any other setting you may need. Discuss and justify your choices in the report.

## 3. (10 points) Training. 
Make plots with the loss function computed over the training set and over the validation set. Stop the training when the error is small enough. Justify your stopping criterium. Report the final accuracy obtained and the confusion matrix on the validation dataset.

## 4. (20 points) Implementation. 
We will run and check the uploaded Python file. To obtain the points for this subproblem, the Python file has to run (no errors) and the MLP model and the Backpropagation algorithm have to be implemented completely from scratch by you. You are not allowed to use any library which implements MLP models, but you are allowed to use auxiliary libraries, e.g. Numpy, Matplotlib, Pandas.

## II. Peer Review paragraph (0 points)
Finally, each group member must write a single paragraph outlining their opinion on the work distri- bution within the group. Did every group member contribute equally? Did you split up tasks in a fair manner, or jointly worked through the exercises. Do you think that some members of your group deserve a different grade from others?